In [ ]:
import os
import argparse
import numpy as np
from PIL import Image
from models import *
import torch
import torch.nn as nn
import torchvision.transforms as tfs
import torchvision.utils as vutils
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
import tqdm
import torch.nn.functional as F
import torchvision.transforms as T
from torchvision.utils import save_image

In [ ]:
totensor = T.ToTensor()
topil = T.ToPILImage()

def clamp(X, lower_limit, upper_limit):
    return torch.max(torch.min(X, upper_limit), lower_limit)

def tensorShow(tensors, titles=['haze']):
    fig = plt.figure()
    for tensor, tit, i in zip(tensors, titles, range(len(tensors))):
        img = make_grid(tensor)
        npimg = img.numpy()
        ax = fig.add_subplot(221+i)
        ax.imshow(np.transpose(npimg, (1, 2, 0)))
        ax.set_title(tit)
    plt.show()


# abs=os.getcwd()+'/'
abs = os.getcwd()+'\\'


dataset = "ots"
gps = 3
blocks = 19



# model_dir=abs+f'trained_models/{dataset}_train_ffa_{gps}_{blocks}.pk'
model_dir = abs + f'trained_models\\{dataset}_train_ffa_{gps}_{blocks}.pk'

device = 'cuda' if torch.cuda.is_available() else 'cpu'
ckp = torch.load(model_dir, map_location=device)
net = FFA(gps=gps, blocks=blocks)
net = nn.DataParallel(net)
net.load_state_dict(ckp['model'])
net.eval()

In [ ]:
cifar10_mean = (0.0, 0.0, 0.0)
cifar10_std = (1.0, 1.0, 1.0)
mu = torch.tensor(cifar10_mean).view(3,1,1).cuda()
std = torch.tensor(cifar10_std).view(3,1,1).cuda()

epsilon = 8
start_epsilon = 8
step_alpha = 2
seed = 160
num_img = 20
attack_iter = 50

upper_limit = ((1 - mu) / std)
lower_limit = ((0 - mu) / std)

epsilon = (epsilon / 255.) / std
start_epsilon = (start_epsilon / 255.) / std
step_alpha = (step_alpha / 255.) / std

image_path = "test_imgs/1400_2.png"
input_image = Image.open(image_path)
input_image = torch.unsqueeze(totensor(input_image).cuda(), 0)
delta = torch.zeros_like(input_image).cuda()
delta.requires_grad = True

clean_image = input_image
# 

In [ ]:
for k in range(attack_iter):
    k += 1
    dehazed_image = net(clean_image + delta)

    # wanna the dehazed output similar to the orginal, which means the model does nothing
    loss = F.mse_loss(clean_image.data, dehazed_image.float())
    loss.backward()
    grad = delta.grad.detach()
    d = delta
    d = clamp(d + step_alpha * torch.sign(grad), -epsilon, epsilon)
    delta.data = d
    delta.grad.zero_()

adv_image = clamp(clean_image + delta, lower_limit, upper_limit)
adv_image = torch.unsqueeze(adv_image.cuda(), 0)
adv_output = net(adv_image)
clean_output = net(input_image)

save_image(torch.cat((adv_output, adv_image, clean_output, input_image),0), "adv_output/" + "zero_output2_" + image_path.split("/")[-1])